In [36]:
import numpy as np
import nibabel as nib
import pydicom as pyd
import argparse
from matplotlib import pyplot as plt
import matplotlib as mpl
import glob as glob
import os
import tqdm 

# Set Variables
precision=np.float32

In [40]:
def CastorToNifti(BaseName):
    
    # Set Header file 
    hdrPath = BaseName+'.hdr'
    # Read Header file

    header = dict()
    with open (hdrPath) as f:
      l = f.readline()
      while l:
        l = f.readline()
        # split keys from values
        s = l.split(':=')
        if len(s)>1:
          # store key-value pair, trim any white spaces
          header[s[0].strip().replace('!','')]=s[1].strip()

    # Initialise dimensions
    # image dimensions (X Y Z)
    nbDim = int(header['number of dimensions'])
    # number of voxels for each dimension
    dimNbVox = np.zeros(nbDim, np.int32)
    # voxel size in mm for each dimension
    dimVoxSize = np.zeros(nbDim, np.float32)
    # offset for the field-of-view in mm
    offset = np.zeros(nbDim, np.float32)

    # Read image 
    for d in range(0,nbDim):
      dimNbVox[d]=int(header['matrix size [{:d}]'.format(d+1)])
      dimVoxSize[d]=float(header['scaling factor (mm/pixel) [{:d}]'.format(d+1)])
      offset[d]=float(header['first pixel offset (mm) [{:d}]'.format(d+1)])

    #Set Image file 

    # Set Header file 
    imgPath = BaseName+'.img'

    # Read image 
    im = np.transpose(np.fromfile(imgPath, dtype=precision, count=-1).reshape(dimNbVox[::-1].tolist()))

    # create the final NIfTI image
    niftiIm = nib.Nifti1Image(im, affine)
    niftiIm.header.set_xyzt_units('mm', 'sec')
    # Save the nifty file 
    nib.save(niftiIm, BaseName+'.nii')


In [41]:
Frames_for_navigator=glob.glob('FramebyFrame_MLEM_ForPANDA/FramebyFrame_MLEM_ForPANDA_it1*.img')

for file in tqdm.tqdm(Frames_for_navigator):
    BaseName=os.path.splitext(file)[0]
    CastorToNifti(BaseName)



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 355/355 [00:37<00:00,  9.47it/s]
